<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/Tensorflow/tf_dataset_tutorial/Exercise_tfds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-rectified-adam

  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=ad2207deb3db0bb4ae7d46189f3924f2b21bfba31aed80fce9ee2133ad7a1d68
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam


In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd ./gdrive/My\ Drive/Deep_learning/Paper_implementation/VGG
!ls

/content/gdrive/My Drive/Deep_learning/Paper_implementation/VGG
checkpoints  Exercise_tfds.ipynb  logs


### Load Library

In [7]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import os, math
from tensorflow.keras import layers, datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from datetime import datetime
from tqdm import tqdm_notebook
from glob import glob
from PIL import Image
from keras_radam.training import RAdamOptimizer
%matplotlib inline
print(tf.__version__)

2.1.0


### Hyperparameter

In [0]:
num_epoch = 5
batch_size = 32
input_shape = (32, 32, 3)
learning_rate = 0.001
dropout_rate = 0.25
num_classes = 10
initial_epoch = 0

loss_ftn = tf.keras.losses.sparse_categorical_crossentropy
metrics = ['accuracy']
# optm = tf.keras.optimizers.Adam(learning_rate)
optm = RAdamOptimizer(learning_rate)

### Model

In [9]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(96, 3, 1, 'SAME', activation='relu')(inputs)
net = layers.MaxPool2D(2)(net)
net = layers.Conv2D(256, 3, 1, 'SAME', activation='relu')(net)
net = layers.MaxPool2D(2)(net)
net = layers.Conv2D(384, 3, 1, 'SAME', activation='relu')(net)
net = layers.Conv2D(384, 3, 1, 'SAME', activation='relu')(net)
net = layers.Conv2D(256, 3, 1, 'SAME', activation='relu')(net)
net = layers.MaxPool2D(2)(net)
net = layers.Flatten()(net)
net = layers.Dense(1024)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(1024)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(10)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='My_AlexNet')
model.compile(optm, loss=loss_ftn, metrics=metrics)
model.summary()

Model: "My_AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 256)       221440    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 384)         885120    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 8, 384)         1

### Callbacks

In [0]:
# Tensorboard
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard = tf.keras.callbacks.TensorBoard(
    logdir,
    histogram_freq=1,
    write_graph=True,
    write_images=True
)

# Learning rate scheduler
def scheduler(epoch):
  if epoch < 10: return 0.001
  else: return 0.001 * math.exp(0.1 * (10 - epoch))
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

# Checkpoints
ckptdir = 'checkpoints'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    ckptdir,
    monitor='val_accuracy',
    verbose=1,
    save_best_inly=True
)

# callbacks = [tensorboard, learning_rate_scheduler, checkpoint]
callbacks = [learning_rate_scheduler]

### Data load

In [11]:
(train_ds, test_ds), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteL0J009/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteL0J009/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


### Define Augmentation ftn

In [0]:
def normalize_img(image, label):
  return tf.cast(image, dtype=tf.float32) / 255., label

def image_preprocess(image, label):
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  image = tf.image.random_brightness(image, 0.2)
  return image, label

### Make dataset

In [0]:
trian_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.map(image_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
train_ds = train_ds.repeat()

test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.repeat()

### Training

In [14]:
steps_per_epoch = ds_info.splits['train'].num_examples // batch_size
validation_steps = ds_info.splits['test'].num_examples // batch_size
steps_per_epoch, validation_steps

(1562, 312)

In [18]:
# history = model.fit(
#     train_ds,
#     epochs=num_epoch,
#     callbacks=callbacks,
#     validation_data=test_ds,
#     initial_epoch=initial_epoch,
#     steps_per_epoch=steps_per_epoch,
#     validation_steps=validation_steps
# )
history = model.fit(
    train_ds,
    epochs=num_epoch,
    validation_data=test_ds,
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

Train for 1562 steps, validate for 312 steps
Epoch 1/5
1562/1562 [==============================] - 37s 23ms/step - loss: 1.7283 - accuracy: 0.3676 - val_loss: 2.6385 - val_accuracy: 0.1020
Epoch 2/5
1562/1562 [==============================] - 20s 13ms/step - loss: 1.3878 - accuracy: 0.5023 - val_loss: 3.5615 - val_accuracy: 0.1002
Epoch 3/5
1562/1562 [==============================] - 20s 13ms/step - loss: 1.2248 - accuracy: 0.5680 - val_loss: 3.8000 - val_accuracy: 0.1061
Epoch 4/5
1562/1562 [==============================] - 21s 13ms/step - loss: 1.1089 - accuracy: 0.6120 - val_loss: 3.7971 - val_accuracy: 0.1052
Epoch 5/5
1562/1562 [==============================] - 20s 13ms/step - loss: 1.0202 - accuracy: 0.6476 - val_loss: 3.7655 - val_accuracy: 0.1148
